In [1]:
# -*- coding: utf-8 -*-
!pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.6'
!pip install 'https://github.com/levan92/deep_sort_realtime.git'
# ================= Monkey‐patch для PIL и numpy ===================
import numpy as np
if not hasattr(np, 'bool'):
    np.bool = bool

from PIL import Image
if not hasattr(Image, 'LINEAR'):
    from PIL.Image import Resampling
    Image.NEAREST  = Resampling.NEAREST
    Image.BILINEAR = Resampling.BILINEAR
    Image.BICUBIC  = Resampling.BICUBIC
    Image.LANCZOS  = Resampling.LANCZOS
    Image.BOX      = Resampling.BOX
    Image.HAMMING  = Resampling.HAMMING
    Image.LINEAR   = Resampling.BILINEAR

# ================= Импорты ===================
import os, random, json, pandas as pd, cv2, matplotlib.pyplot as plt
import torch
from torch.utils.tensorboard import SummaryWriter


  Cloning https://github.com/facebookresearch/detectron2.git (to revision v0.6) to /tmp/pip-req-build-iczaikfd
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-iczaikfd
  Running command git checkout -q d1e04565d3bec8719335b88be9e9b961bf3ec464
  Resolved https://github.com/facebookresearch/detectron2.git to commit d1e04565d3bec8719335b88be9e9b961bf3ec464
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.7 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp311-cp311-linux_x86_64.whl size=6424844 sha256=d48d4bfd734be2b8beeb53ff94d2a475b7fcb0da7b8e7399e991a9cee6b

/tmp/ipykernel_35/2732540068.py:5: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, 'bool'):


In [2]:
# Detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, DefaultPredictor, HookBase, EvalHook
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.data import detection_utils as utils
from detectron2.data.datasets import load_coco_json
from detectron2.data.dataset_mapper import DatasetMapper
import detectron2.data.transforms as T
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.utils.logger import setup_logger

import math

In [3]:
# ================= Параметры пути ===================
DATA_ROOT   = '/kaggle/working/wheels5k-coco/train'
IMG_DIR     = '/kaggle/input/wheels5k-coco/train'
DS_DIR      = os.path.join('/kaggle/working/wheels5k-coco', 'datasets_filtered')  # JSON после remap category_id->0
OUT_ROOT    = os.path.join('/kaggle/working/', 'output')
os.makedirs(OUT_ROOT, exist_ok=True)

# TensorBoard и CSV
tb_writer = SummaryWriter(log_dir=os.path.join(OUT_ROOT, 'tensorboard'))
CSV_PATH  = os.path.join(OUT_ROOT, 'metrics.csv')



In [4]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
from pycocotools import mask as mask_utils

coco_path = '/kaggle/input/wheels5k-coco/train/_annotations.coco.json'
images_dir = '/kaggle/input/wheels5k-coco/train'

with open(coco_path, 'r') as f:
    coco = json.load(f)

def filter_by_tag(imgs, tag):
    return [img for img in imgs
            if 'extra' in img 
            and 'user_tags' in img['extra'] 
            and tag in img['extra']['user_tags']]

start_imgs = filter_by_tag(coco['images'], 'start')
end_imgs   = filter_by_tag(coco['images'], 'end')

def split(imgs):
    tr, v = train_test_split(imgs, test_size=0.2, random_state=42)
    v, te = train_test_split(v, test_size=0.5, random_state=42)
    return tr, v, te

train_start, val_start, test_start = split(start_imgs)
train_end,   val_end,   test_end   = split(end_imgs)

test_imgs = test_start + test_end

def filter_anns(imgs):
    ids = set(i['id'] for i in imgs)
    return [a for a in coco['annotations'] if a['image_id'] in ids]

splits = {
    'test': (test_imgs, filter_anns(test_imgs)),
    'val_start':   (val_start,   filter_anns(val_start)),
    'val_end':   (val_end,   filter_anns(val_end)),
    'train_start': (train_start, filter_anns(train_start)),
    'train_end':   (train_end,   filter_anns(train_end))
}

os.makedirs('/kaggle/working/wheels5k-coco/datasets1', exist_ok=True)
for name, (imgs, anns) in splits.items():
    out = {'images': imgs, 'annotations': anns, 'categories': coco['categories']}
    with open(f'/kaggle/working/wheels5k-coco/datasets1/{name}.json','w') as f:
        json.dump(out, f)


In [5]:
import json, os

def filter_and_remap_coco(input_json, output_json, target_cat_id=6):
    data = json.load(open(input_json))
    # Оставляем только нужные category и ann
    new_anns = []
    img_ids = set()
    for ann in data['annotations']:
        if ann['category_id'] == target_cat_id:
            ann['category_id'] = 0   # Remap to 0
            new_anns.append(ann)
            img_ids.add(ann['image_id'])
    # Фильтруем изображения
    new_imgs = [img for img in data['images'] if img['id'] in img_ids]
    # Новое описание категории
    new_cats = [{"id": 0, "name": "wheel"}]
    # Сохраняем
    out = {
        "images": new_imgs,
        "annotations": new_anns,
        "categories": new_cats
    }
    json.dump(out, open(output_json, "w"), indent=2)
    print(f"Filtered {len(data['annotations'])}->{len(new_anns)} annotations, images {len(data['images'])}->{len(new_imgs)}")

# Примените к вашим файлам
os.makedirs("/kaggle/working/wheels5k-coco/datasets_filtered", exist_ok=True)
filter_and_remap_coco("/kaggle/working/wheels5k-coco/datasets1/train_start.json", "/kaggle/working/wheels5k-coco/datasets_filtered/start_train.json")
filter_and_remap_coco("/kaggle/working/wheels5k-coco/datasets1/val_start.json",   "/kaggle/working/wheels5k-coco/datasets_filtered/start_val.json")
filter_and_remap_coco("/kaggle/working/wheels5k-coco/datasets1/train_end.json", "/kaggle/working/wheels5k-coco/datasets_filtered/end_train.json")
filter_and_remap_coco("/kaggle/working/wheels5k-coco/datasets1/val_end.json",   "/kaggle/working/wheels5k-coco/datasets_filtered/end_val.json")

Filtered 5365->3416 annotations, images 1728->1437
Filtered 648->432 annotations, images 216->178
Filtered 4980->3293 annotations, images 1828->1506
Filtered 619->426 annotations, images 228->193


In [6]:
# ================= Регистрация датасетов ===================
def register_split(name, json_file):
    DatasetCatalog.register(name, lambda: load_coco_json(json_file, IMG_DIR))
    MetadataCatalog.get(name).set(thing_classes=['wheel'], evaluator_type='coco')

# Сплиты train/val для start и end (используют один и тот же remapped JSON)

#register_split('t_end',   os.path.join(DS_DIR, 'end_train.json'))
#register_split('v_end',   os.path.join(DS_DIR, 'end_val.json'))
register_split('_t_start', os.path.join(DS_DIR, 'start_train.json'))
register_split('_v_start', os.path.join(DS_DIR, 'start_val.json'))


In [7]:
# ================= Функция генерации конфига ===================
def make_cfg(train_ds, val_ds, output_dir,
             max_iter=16160, base_lr=1e-3, batch_size=4):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"
    ))
    # Датасеты
    cfg.DATASETS.TRAIN        = (train_ds,)
    cfg.DATASETS.TEST         = (val_ds,)
    # Dataloader
    cfg.DATALOADER.NUM_WORKERS = 4
    cfg.SOLVER.IMS_PER_BATCH   = batch_size
    # LR-стратегия
    cfg.SOLVER.BASE_LR         = base_lr
    cfg.SOLVER.WEIGHT_DECAY    = 1e-4
    cfg.SOLVER.WARMUP_ITERS    = 800
    cfg.SOLVER.WARMUP_METHOD   = "linear"
    cfg.SOLVER.MAX_ITER        = max_iter
    cfg.SOLVER.STEPS           = (int(max_iter * 0.6), int(max_iter * 0.9))
    cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
    cfg.TEST.EVAL_PERIOD       = 377  # валидация каждые 500 итераций
    

    # ROI Heads
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 48
    cfg.INPUT.MAX_SIZE_TRAIN = 1024
    cfg.INPUT.MAX_SIZE_TEST  = 1024
    # Вывод
    cfg.OUTPUT_DIR = output_dir
    os.makedirs(output_dir, exist_ok=True)

    cfg.MODEL.DEVICE = "cuda"
    return cfg



In [8]:
class BestCheckpointer(HookBase):
    def __init__(self, checkpointer, eval_period, save_name="model_best"):
        self.checkpointer = checkpointer
        self.eval_period  = eval_period
        self.best_metric  = -float("inf")
        self.save_name    = save_name

    def after_step(self):
        it = self.trainer.iter
        if it == 0 or (it + 1) % self.eval_period != 0:
            return
        latest = self.trainer.storage.latest()
        if "segm/AP50" not in latest:
            return
        current = latest["segm/AP50"][0]
        if current > self.best_metric:
            self.best_metric = current
            self.checkpointer.save(self.save_name)
            print(f"[BestCheckpointer] New best segm/AP50 = {current:.4f} at iter {it+1}")




In [9]:
import torch
import detectron2.utils.comm as comm
from detectron2.engine import HookBase
from detectron2.data import build_detection_test_loader
from detectron2.data.dataset_mapper import DatasetMapper

class ValidationLossHook(HookBase):
    def __init__(self, eval_period, trainer):
        """
        eval_period: сколько шагов между запуском валидации
        trainer:     экземпляр вашего MyTrainer
        """
        self._period  = eval_period
        self.trainer  = trainer
        # сборка val-loader, но с is_train=True, чтобы были gt_instances
        cfg = trainer.cfg
        mapper = DatasetMapper(cfg, is_train=True, augmentations=[])
        self._data_loader = iter(
            build_detection_test_loader(cfg, cfg.DATASETS.TEST[0], mapper=mapper)
        )

    def _get_val_loss(self):
        # получаем следующий батч
        try:
            data = next(self._data_loader)
        except StopIteration:
            # если даталоадер кончился, пересоздаём
            cfg    = self.trainer.cfg
            mapper = DatasetMapper(cfg, is_train=True, augmentations=[])
            self._data_loader = iter(
                build_detection_test_loader(cfg, cfg.DATASETS.TEST[0], mapper=mapper)
            )
            data = next(self._data_loader)

        # вычисляем лосс
        with torch.no_grad():
            loss_dict = self.trainer.model(data)  # training mode → вернёт dict losses
            # суммируем и редуцируем по всем устройствам
            losses_reduced = comm.reduce_dict(loss_dict)
            total_val_loss = sum(v.item() for v in losses_reduced.values())
            # префикс 'val_' для всех скаляров
            loss_dict_reduced = {
                f"val_{k}": v.item() for k, v in losses_reduced.items()
            }

        return total_val_loss, loss_dict_reduced

    def after_step(self):
        next_iter = self.trainer.iter + 1
        # запускаем каждые eval_period шагов и на последнем
        if next_iter % self._period != 0 and next_iter != self.trainer.max_iter:
            return
        total_val_loss, loss_dict_reduced = self._get_val_loss()
        if comm.is_main_process():
            # записываем в storage, чтобы их писал Tensorboard/X
            self.trainer.storage.put_scalar("total_val_loss", total_val_loss)
            self.trainer.storage.put_scalars(**loss_dict_reduced)

In [10]:
# ================= Custom Trainer ===================
from detectron2.engine import hooks as d2_hooks
from detectron2.data import build_detection_train_loader

class MyTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        # 1) Считаем, во сколько пикселей надо уменьшить,
        #    взяв оригинальный диапазон MIN_SIZE_TRAIN и разделив на 2.5
        #min_size   = cfg.INPUT.MIN_SIZE_TRAIN
        #max_size   = cfg.INPUT.MAX_SIZE_TRAIN
        #scale      = 2
        #new_min    = max(1, math.floor(min_size/scale))
        #new_max    = max(1, math.floor(max_size/scale))

        # 2) Формируем список аугментаций, начиная с простого ресайза
        aug_list = [
        #    T.ResizeShortestEdge(new_min, new_max, sample_style="choice"),
        ]
        mapper = DatasetMapper(cfg, is_train=True, augmentations=aug_list)
        return build_detection_train_loader(cfg, mapper=mapper)

    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        # ТО-ЖЕ самое для валидации / инференса
        #min_size = cfg.INPUT.MIN_SIZE_TEST
        #max_size = cfg.INPUT.MAX_SIZE_TEST
        #scale    = 2.5
        #new_min  = max(1, math.floor(min_size/scale))
        #new_max  = max(1, math.floor(max_size/scale))

        aug_list = [
        #    T.ResizeShortestEdge([new_min], new_max, sample_style="choice"),
        ]
        mapper = DatasetMapper(cfg, is_train=False, augmentations=aug_list)
        return build_detection_test_loader(cfg, dataset_name, mapper=mapper)
        
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        evaluator = COCOEvaluator(
        dataset_name,
        tasks=["bbox","segm"],
        distributed=False,
        output_dir=cfg.OUTPUT_DIR,
        )
        return evaluator

    @classmethod
    def build_optimizer(cls, cfg, model):
        lr = cfg.SOLVER.BASE_LR
        # Собираем все параметры backbone
        backbone_set = set(model.backbone.parameters())
        # Две группы: backbone и всё остальное
        backbone_params = []
        other_params    = []
        for p in model.parameters():
            if not p.requires_grad:
                continue
            if p in backbone_set:
                backbone_params.append(p)
            else:
                other_params.append(p)
        optimizer = torch.optim.SGD(
            [
              {"params": backbone_params, "lr": lr * 0.1},
              {"params": other_params,    "lr": lr}
            ],
            lr=lr, momentum=0.9, weight_decay=cfg.SOLVER.WEIGHT_DECAY
        )
        return optimizer

    def build_hooks(self):
        hooks_list = super().build_hooks()
        insertion_idx = len(hooks_list) - 1
        # вставляем наш ValidationLossHook
        val_hook = ValidationLossHook(
            eval_period=self.cfg.TEST.EVAL_PERIOD,
            trainer=self
        )
        hooks_list.insert(insertion_idx, val_hook)
        # Вставляем после EvalHook
        for idx, h in enumerate(hooks_list):
            if isinstance(h, EvalHook):
                checkpointer = DetectionCheckpointer(self.model, self.cfg.OUTPUT_DIR)
                best_cp = BestCheckpointer(checkpointer,
                                           eval_period=self.cfg.TEST.EVAL_PERIOD,
                                           save_name="model_best")
                hooks_list.insert(idx+1, best_cp)
                break
        return hooks_list


In [11]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [12]:
# ================= Main: тренировка start и end ===================
setup_logger()

# --- Start split ---
cfg_s = make_cfg("_t_start", "_v_start", os.path.join(OUT_ROOT, "start"), max_iter=16160)
trainer_s = MyTrainer(cfg_s)
trainer_s.resume_or_load(resume=False)
trainer_s.train()




[05/15 22:09:55 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

R-50.pkl: 102MB [00:00, 163MB/s]                             


[05/15 22:09:57 d2.checkpoint.c2_model_loading]: Renaming Caffe2 weights ......
[05/15 22:09:57 d2.checkpoint.c2_model_loading]: Following weights matched with submodule backbone.bottom_up:
| Names in Model    | Names in Checkpoint      | Shapes                                          |
|:------------------|:-------------------------|:------------------------------------------------|
| res2.0.conv1.*    | res2_0_branch2a_{bn_*,w} | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| res2.0.conv2.*    | res2_0_branch2b_{bn_*,w} | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| res2.0.conv3.*    | res2_0_branch2c_{bn_*,w} | (256,) (256,) (256,) (256,) (256,64,1,1)        |
| res2.0.shortcut.* | res2_0_branch1_{bn_*,w}  | (256,) (256,) (256,) (256,) (256,64,1,1)        |
| res2.1.conv1.*    | res2_1_branch2a_{bn_*,w} | (64,) (64,) (64,) (64,) (64,256,1,1)            |
| res2.1.conv2.*    | res2_1_branch2b_{bn_*,w} | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| res2.1.conv3.*  

/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/15 22:10:22 d2.utils.events]:  eta: 4:53:55  iter: 19  total_loss: 4.7  loss_cls: 3.263  loss_box_reg: 0.005213  loss_mask: 0.693  loss_rpn_cls: 0.6923  loss_rpn_loc: 0.06252  time: 1.1052  data_time: 0.0273  lr: 2.4583e-06  max_mem: 10504M
[05/15 22:10:44 d2.utils.events]:  eta: 4:53:27  iter: 39  total_loss: 1.817  loss_cls: 0.3811  loss_box_reg: 0.00306  loss_mask: 0.6852  loss_rpn_cls: 0.6695  loss_rpn_loc: 0.06008  time: 1.1022  data_time: 0.0151  lr: 4.9407e-06  max_mem: 10504M
[05/15 22:11:06 d2.utils.events]:  eta: 4:53:33  iter: 59  total_loss: 1.814  loss_cls: 0.437  loss_box_reg: 0.02745  loss_mask: 0.6708  loss_rpn_cls: 0.6167  loss_rpn_loc: 0.06114  time: 1.1032  data_time: 0.0145  lr: 7.4231e-06  max_mem: 10504M
[05/15 22:11:28 d2.utils.events]:  eta: 4:53:43  iter: 79  total_loss: 1.839  loss_cls: 0.5178  loss_box_reg: 0.07246  loss_mask: 0.657  loss_rpn_cls: 0.5425  loss_rpn_loc: 0.06311  time: 1.1039  data_time: 0.0150  lr: 9.9055e-06  max_mem: 10505M
[05/15 22:11:

In [3]:
# ================= Оценка и лог в CSV + TensorBoard ===================
def evaluate_and_log(trainer, cfg, split, tag):
    evaluator  = COCOEvaluator(split, cfg, False, output_dir=cfg.OUTPUT_DIR)
    val_loader = build_detection_test_loader(cfg, split)
    metrics    = inference_on_dataset(trainer.model, val_loader, evaluator)
    segm       = metrics.get("segm", {})
    row = {
        "tag":     tag,
        "AP50:95": segm.get("AP", 0),
        "AP50":    segm.get("AP50",0),
        "AP75":    segm.get("AP75",0)
    }
    # CSV
    df = pd.DataFrame([row])
    df.to_csv(CSV_PATH, mode='a', header=not os.path.exists(CSV_PATH), index=False)
    # TensorBoard
    for k,v in row.items():
        if k!="tag": tb_writer.add_scalar(f"val/{tag}/{k}", v)
    return metrics

metrics_s = evaluate_and_log(trainer_s, cfg_s, "v_start", "start")
#metrics_e = evaluate_and_log(trainer_e, cfg_e, "v_end",   "end")

# ================= Инференс и визуализация 4 примеров ===================
predictor_s = DefaultPredictor(cfg_s)
#predictor_e = DefaultPredictor(cfg_e)
meta_val    = MetadataCatalog.get("v_start")  # общий метадатасет
ds_val      = DatasetCatalog.get("v_start")

for d in random.sample(ds_val, 4):
    img = cv2.imread(os.path.join(IMG_DIR, d["file_name"]))
    tag = "start" 
    pred = (predictor_s)(img)
    vis  = Visualizer(img[:,:,::-1], metadata=meta_val, scale=0.8)
    out  = vis.draw_instance_predictions(pred["instances"].to("cpu"))
    plt.figure(figsize=(8,6))
    plt.imshow(out.get_image()[:,:,::-1])
    plt.title(f"{tag} | {d['file_name']}")
    plt.axis("off")
    plt.show()

print("Пайплайн завершён! Метрики сохранены в", CSV_PATH)


NameError: name 'trainer_s' is not defined

In [ ]:
# trainer = ваш объект DefaultTrainer или его наследник
# cfg - ваш конфиг, cfg.OUTPUT_DIR указывает папку для чекпоинтов

checkpointer = DetectionCheckpointer(trainer.model, save_dir=cfg.OUTPUT_DIR)  # [1]
checkpointer.save("model_final")  # сохранит файл OUTPUT_DIR/model_final.pth  [1]

In [ ]:
# model - тот же состав модели, что и при обучении
checkpointer_ = DetectionCheckpointer(model, save_dir=(f'cfg.OUTPUT_DIR'+'/start')
checkpointer.load("model_best.pth")  # загрузит веса в model  [2]

In [ ]:
# --- End split ---
setup_logger()

cfg_e = make_cfg("t_end", "v_end", os.path.join(OUT_ROOT, "end"), max_iter=18850)
trainer_e = MyTrainer(cfg_e)
trainer_e.resume_or_load(resume=True)
trainer_e.train()

In [3]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.6'

  Cloning https://github.com/facebookresearch/detectron2.git (to revision v0.6) to /tmp/pip-req-build-4bkn3osc
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-4bkn3osc
  Running command git checkout -q d1e04565d3bec8719335b88be9e9b961bf3ec464
  Resolved https://github.com/facebookresearch/detectron2.git to commit d1e04565d3bec8719335b88be9e9b961bf3ec464
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.1 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp311-cp311-linux_x86_64.whl size=6424859 sha256=4d9cde59e8596d2dc96cbce04c7986e6dba52a3b07b884a7c791d7948e3

In [ ]:
import cv2
import torch
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2 import model_zoo
from detectron2.checkpoint import DetectionCheckpointer

# 1. Инициализация модели
def setup_config(model_weights_path, config_path, confidence_threshold=0.7):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"
    ))
    cfg.MODEL.WEIGHTS = model_weights_path
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = confidence_threshold
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    return cfg

# 2. Обработка видео
def process_video(input_path, output_path, cfg):
    # Инициализация видеопотоков
    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Инициализация VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Инициализация модели и метаданных
    predictor = DefaultPredictor(cfg)
    metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0] if cfg.DATASETS.TRAIN else "__unused")
    
    # Счетчик объектов
    counts = {}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Инференс
        outputs = predictor(frame)

        # Визуализация
        v = Visualizer(frame[:, :, ::-1], metadata=metadata, scale=1.2)
        instances = outputs["instances"].to("cpu")
        vis_frame = v.draw_instance_predictions(instances).get_image()[:, :, ::-1]

        # Подсчет объектов
        for cid in instances.pred_classes:
            class_name = metadata.thing_classes[cid]
            counts[class_name] = counts.get(class_name, 0) + 1

        # Запись кадра
        out.write(vis_frame)

    # Освобождение ресурсов
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    # Вывод результатов
    print("Обнаруженные объекты:")
    for name, count in counts.items():
        print(f"{name}: {count}")


            # сохраняем для рисования
    prev_masks = []
    for mask, tobj, cy in zip(masks, tracked, centroids):
        prev_masks.append({
            "mask": mask,
            "id":    tobj.id,
            "cy":    cy
            })

    prev_gray = gray.copy()
# Пример использования
if __name__ == "__main__":
    config_path = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"  # Путь к вашему конфигу
    model_weights = "start/model_best.pth"  # Путь к вашим весам
    input_video = "test1.avi"
    output_video = "output!.avi"

    cfg = setup_config(model_weights, config_path)
    process_video(input_video, output_video, cfg)

In [5]:
import numpy as np
if not hasattr(np, 'bool'):
    np.bool = bool

from PIL import Image
if not hasattr(Image, 'LINEAR'):
    from PIL.Image import Resampling
    Image.NEAREST  = Resampling.NEAREST
    Image.BILINEAR = Resampling.BILINEAR
    Image.BICUBIC  = Resampling.BICUBIC
    Image.LANCZOS  = Resampling.LANCZOS
    Image.BOX      = Resampling.BOX
    Image.HAMMING  = Resampling.HAMMING
    Image.LINEAR   = Resampling.BILINEAR


/tmp/ipykernel_35/2433867498.py:2: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, 'bool'):


In [14]:
import cv2
import numpy as np
from collections import OrderedDict
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo
import time


# ---------------------------
# Класс простого Калман-фильтра
# ---------------------------
class KalmanFilter2D:
    def __init__(self, dt=1.0, process_var=1e-2, meas_var=1e-1):
        self.dt = dt
        self.A = np.array([[1, 0, dt, 0], [0, 1, 0, dt], [0, 0, 1, 0], [0, 0, 0, 1]], dtype=float)
        self.H = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], dtype=float)
        self.Q = process_var * np.eye(4)
        self.R = meas_var * np.eye(2)
        self.P = np.eye(4)
        self.x = np.zeros((4,1))
        self.inited = False

    def predict(self):
        self.x = self.A @ self.x
        self.P = self.A @ self.P @ self.A.T + self.Q
        return self.x[:2].ravel()

    def correct(self, meas):
        z = np.array(meas, dtype=float).reshape((2,1))
        if not self.inited:
            self.x[:2] = z
            self.inited = True
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        y = z - (self.H @ self.x)
        self.x = self.x + K @ y
        I = np.eye(self.P.shape[0])
        self.P = (I - K @ self.H) @ self.P
        return self.x[:2].ravel()


# ---------------------------
# Трекер на основе центроидов + Калман
# ---------------------------
class KalmanCentroidTracker:
    def __init__(self, max_disappeared=5, max_distance=50, dt=1.0):
        self.next_id = 0
        self.tracks = OrderedDict()
        self.disappeared = OrderedDict()
        self.max_disappeared = max_disappeared
        self.max_distance = max_distance
        self.dt = dt

    def register(self, centroid, bottom, contour):
        kf = KalmanFilter2D(dt=self.dt)
        kf.correct(centroid)
        self.tracks[self.next_id] = {
            'kf': kf,
            'centroids': [centroid],
            'bottoms': [bottom],
            'contours': [contour],
            'predicted': []
        }
        self.disappeared[self.next_id] = 0
        self.next_id += 1

    def deregister(self, track_id):
        del self.tracks[track_id]
        del self.disappeared[track_id]

    def update(self, detections):
        if len(detections) == 0:
            for tid, tr in list(self.tracks.items()):
                pred = tr['kf'].predict()
                tr['predicted'].append(tuple(pred))
                self.disappeared[tid] += 1
                if self.disappeared[tid] > self.max_disappeared:
                    self.deregister(tid)
            return self.tracks

        input_centroids = [d['centroid'] for d in detections]
        input_bottoms = [d['bottom'] for d in detections]
        input_contours = [d['contour'] for d in detections]

        if len(self.tracks) == 0:
            for i in range(len(detections)):
                self.register(input_centroids[i], input_bottoms[i], input_contours[i])
        else:
            track_ids = list(self.tracks.keys())
            prev_states = [self.tracks[tid]['kf'].predict() for tid in track_ids]
            D = np.linalg.norm(np.array(prev_states)[:, None, :] - np.array(input_centroids)[None, :, :], axis=2)
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]

            assigned_rows, assigned_cols = set(), set()
            for r, c in zip(rows, cols):
                if r in assigned_rows or c in assigned_cols or D[r, c] > self.max_distance:
                    continue
                tid = track_ids[r]
                corrected = self.tracks[tid]['kf'].correct(input_centroids[c])
                self.tracks[tid]['centroids'].append(tuple(corrected))
                self.tracks[tid]['bottoms'].append(input_bottoms[c])
                self.tracks[tid]['contours'].append(input_contours[c])
                self.tracks[tid]['predicted'].append(tuple(corrected))
                self.disappeared[tid] = 0
                assigned_rows.add(r)
                assigned_cols.add(c)

            for r in set(range(len(prev_states))) - assigned_rows:
                tid = track_ids[r]
                pred = self.tracks[tid]['kf'].predict()
                self.tracks[tid]['predicted'].append(tuple(pred))
                self.disappeared[tid] += 1
                if self.disappeared[tid] > self.max_disappeared:
                    self.deregister(tid)

            for c in set(range(len(input_centroids))) - assigned_cols:
                self.register(input_centroids[c], input_bottoms[c], input_contours[c])

        return self.tracks


# ---------------------------
# Настройка Detectron2
# ---------------------------

def setup_predictor(weights_path: str, score_thresh: float = 0.5):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"
    ))
    cfg.MODEL.WEIGHTS = weights_path
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = score_thresh
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.MODEL.DEVICE = "cuda"
    return DefaultPredictor(cfg)


def extract_wheel_detections(predictor, frame):
    outputs = predictor(frame)
    masks = outputs['instances'].pred_masks.cpu().numpy()
    dets = []
    for mask in masks:
        m = (mask * 255).astype('uint8')
        cnts, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not cnts:
            continue
        cnt = max(cnts, key=cv2.contourArea)
        M = cv2.moments(cnt)
        if M['m00'] == 0:
            continue
        cx, cy = int(M['m10']/M['m00']), int(M['m01']/M['m00'])
        bottom = tuple(cnt[cnt[:,:,1].argmax()][0])
        dets.append({'centroid':(cx,cy), 'bottom':bottom, 'contour':cnt})
    return dets


# ---------------------------
# Основная функция
# ---------------------------

def main(video_path, weights_path, output_path,
         skip_frames=5, movement_thresh=2.0):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    predictor = setup_predictor(weights_path)
    tracker = KalmanCentroidTracker(max_disappeared=5,
                                    max_distance=50,
                                    dt=skip_frames/fps)

    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    writer = cv2.VideoWriter(output_path, fourcc, fps/skip_frames, (width, height))

    frame_idx = 0
    decel_frame = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1

        if frame_idx % skip_frames == 0:
            dets = extract_wheel_detections(predictor, frame)
        else:
            dets = []

        tracks = tracker.update(dets)

        for tid, tr in tracks.items():
            cx_f, cy_f = tr['centroids'][-1]
            cx, cy = int(cx_f), int(cy_f)
            cnt = tr['contours'][-1]
            cv2.drawContours(frame, [cnt], -1, (0,255,0), 2)
            cv2.putText(frame, f"ID {tid}", (cx, cy-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
            if decel_frame is None and len(tr['centroids'])>=2:
                x1,y1 = tr['centroids'][-2]
                x2,y2 = tr['centroids'][-1]
                if np.hypot(x2-x1, y2-y1) < movement_thresh:
                    decel_frame = frame_idx
                    decel_time = decel_frame / fps
        writer.write(frame)

    cap.release()
    writer.release()

    if decel_frame is not None:
        print(f"Time to deceleration: {decel_time:.2f} s")
    else:
        print("Deceleration not detected.")


In [15]:

main('/kaggle/input/videos/test1.avi', '/kaggle/input/start/other/default/1/model_best.pth', 'out.avi')

Time to deceleration: 1.60 s


In [37]:
import cv2
import numpy as np
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from deep_sort_realtime.deepsort_tracker import DeepSort

def setup_detectron2(weights_path: str, score_thresh: float = 0.5):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"
    ))
    cfg.MODEL.WEIGHTS = weights_path
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = score_thresh
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # колесо
    cfg.MODEL.DEVICE = "cuda"
    predictor = DefaultPredictor(cfg)
    metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
    return predictor, metadata

def setup_predictor(weights_path: str, score_thresh: float = 0.5):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"
    ))
    cfg.MODEL.WEIGHTS = weights_path
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = score_thresh
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
    cfg.MODEL.DEVICE = "cuda"
    return DefaultPredictor(cfg)


class BoundingBoxAnnotator:
    def __init__(self, color=(0, 255, 0), thickness=2):
        self.color = color
        self.thickness = thickness

    def annotate(self, frame, detections):
        for det in detections:
            x1, y1, x2, y2 = det['box']
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), self.color, self.thickness)
        return frame


class LabelAnnotator:
    def __init__(self, color=(0, 255, 0), font_scale=0.6, thickness=2):
        self.color = color
        self.font_scale = font_scale
        self.thickness = thickness

    def annotate(self, frame, detections):
        for det in detections:
            tid = det['track_id']
            x1, y1, _, _ = det['box']
            cv2.putText(
                frame,
                f"Wheel #{tid}",
                (int(x1), int(y1) - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                self.font_scale,
                self.color,
                self.thickness
            )
        return frame


class MaskAnnotator:
    def __init__(self, color=(0, 255, 0), alpha=0.3):
        self.color = np.array(color, dtype=np.uint8)
        self.alpha = alpha

    def annotate(self, frame, detections):
        overlay = frame.copy()
        for det in detections:
            mask = det['mask']
            rgba = np.zeros_like(frame, dtype=np.uint8)
            rgba[mask] = self.color
            overlay = cv2.addWeighted(overlay, 1.0, rgba, self.alpha, 0)
        return overlay


class TraceAnnotator:
    def __init__(self, trace_length=30, color=(0,255,0), thickness=2):
        self.trace_length = trace_length
        self.color = color
        self.thickness = thickness
        self.traces = {}  # track_id -> list of centroids

    def annotate(self, frame, detections):
        for det in detections:
            tid = det['track_id']
            c = det['centroid']
            if tid not in self.traces:
                self.traces[tid] = []
            self.traces[tid].append(c)
            if len(self.traces[tid]) > self.trace_length:
                self.traces[tid].pop(0)
            pts = self.traces[tid]
            for i in range(1, len(pts)):
                cv2.line(frame, tuple(map(int, pts[i-1])), tuple(map(int, pts[i])), self.color, self.thickness)
        return frame


class PolygonZone:
    def __init__(self, polygon):
        self.polygon = np.array(polygon, dtype=np.int32)

    def contains(self, point):
        return cv2.pointPolygonTest(self.polygon, tuple(point), False) >= 0


class PolygonZoneAnnotator:
    def __init__(self, polygon_zone: PolygonZone, color=(0,0,255), thickness=2, opacity=0.1):
        self.zone = polygon_zone
        self.color = color
        self.thickness = thickness
        self.opacity = opacity

    def annotate(self, frame, detections=None):
        overlay = frame.copy()
        cv2.polylines(frame, [self.zone.polygon], True, self.color, self.thickness)
        cv2.fillPoly(overlay, [self.zone.polygon], self.color)
        # полупрозрачность
        frame = cv2.addWeighted(overlay, self.opacity, frame, 1 - self.opacity, 0)
        return frame


def process_video(source_path: str, target_path: str, predictor, metadata,
                  model_tracker, bbox_annotator, mask_annotator,
                  label_annotator, trace_annotator,
                  zone_annotator=None, skip_frames=1,
                  output_fps=25.0):
    cap = cv2.VideoCapture(source_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    writer = cv2.VideoWriter(target_path, fourcc, output_fps, (width, height))

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1
        if frame_idx % skip_frames != 0:
            continue

        # Детекция
        outputs = predictor(frame)
        instances = outputs['instances'].to('cpu')
        boxes = instances.pred_boxes.tensor.numpy()
        masks = instances.pred_masks.numpy()
        scores = instances.scores.numpy()

        # Подготовка детекций для трекера
        dets_for_tracker = []
        for box, score in zip(boxes, scores):
            x1, y1, x2, y2 = box.astype(int)
            dets_for_tracker.append(([x1, y1, x2-x1, y2-y1], score, 'wheel'))

        tracks = model_tracker.update_tracks(dets_for_tracker, frame=frame)

        # Собираем информацию по трекам
        det_list = []
        for track in tracks:
            if not track.is_confirmed():
                continue
            tid = track.track_id
            ltrb = track.to_ltrb()
            mask = None  # сегментацию можно наложить отдельно
            centroid = ((ltrb[0]+ltrb[2])/2, (ltrb[1]+ltrb[3])/2)
            det_list.append({
                'track_id': tid,
                'box': ltrb,
                'centroid': centroid,
                'score': track.det_conf,
                'mask': mask
            })

        # Аннотирование
        annotated = frame.copy()
        if mask_annotator:
            annotated = mask_annotator.annotate(annotated, det_list)
        annotated = bbox_annotator.annotate(annotated, det_list)
        annotated = label_annotator.annotate(annotated, det_list)
        if zone_annotator:
            annotated = zone_annotator.annotate(annotated)
        annotated = trace_annotator.annotate(annotated, det_list)

        writer.write(annotated)

    cap.release()
    writer.release()



    

In [39]:
predictor, metadata = setup_detectron2(
        weights_path="/kaggle/input/end/other/default/1/model_best.pth",
        score_thresh=0.7
    )
    # Используем deepSORT
model_tracker = DeepSort(
        max_age=150,
        n_init=10,
        max_cosine_distance=0.2,
        nn_budget=None
    )

bbox_annotator = BoundingBoxAnnotator()
mask_annotator = MaskAnnotator()
label_annotator = LabelAnnotator()
trace_annotator = TraceAnnotator(trace_length=30)

    # Полигон платформы
polygon = np.array([[464,371],[1465,795],[1919,465],[1919,181],[1879,169]])
polygon_zone = PolygonZone(polygon)
zone_annotator = PolygonZoneAnnotator(polygon_zone)

process_video(
        source_path="/kaggle/input/videos/testup1.avi",
        target_path="output.avi",
        predictor=predictor,
        metadata=metadata,
        model_tracker=model_tracker,
        bbox_annotator=bbox_annotator,
        mask_annotator=mask_annotator,
        label_annotator=label_annotator,
        trace_annotator=trace_annotator,
        zone_annotator=zone_annotator,
        skip_frames=1,
        output_fps=25.0
    )


In [35]:
config_path = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"  # Путь к вашему конфигу
model_weights = "/kaggle/input/end/other/default/1/model_best.pth"  # Путь к вашим весам
input_video = "/kaggle/input/videos/testup1.avi"
output_video = "output.avi"

cfg = setup_config(model_weights, config_path)
process_video(input_video, output_video, cfg)

AttributeError: 'DefaultPredictor' object has no attribute 'DATASETS'

In [2]:
import numpy as np
if not hasattr(np, 'bool'):
    np.bool = bool

from PIL import Image
if not hasattr(Image, 'LINEAR'):
    from PIL.Image import Resampling
    Image.NEAREST  = Resampling.NEAREST
    Image.BILINEAR = Resampling.BILINEAR
    Image.BICUBIC  = Resampling.BICUBIC
    Image.LANCZOS  = Resampling.LANCZOS
    Image.BOX      = Resampling.BOX
    Image.HAMMING  = Resampling.HAMMING
    Image.LINEAR   = Resampling.BILINEAR


/tmp/ipykernel_35/2433867498.py:2: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, 'bool'):


In [3]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.6'
#!pip install 'https://github.com/levan92/deep_sort_realtime.git'
!pip install deep_sort_realtime

  Cloning https://github.com/facebookresearch/detectron2.git (to revision v0.6) to /tmp/pip-req-build-p2wc4_o_
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-p2wc4_o_
  Running command git checkout -q d1e04565d3bec8719335b88be9e9b961bf3ec464
  Resolved https://github.com/facebookresearch/detectron2.git to commit d1e04565d3bec8719335b88be9e9b961bf3ec464
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.8 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp311-cp311-linux_x86_64.whl size=6424836 sha256=b9be11c4c68ae06644b872514517b1d8bfcc0542dbdcee7451bb21fa6766

РАБОТАЕТ НА МАСКИ И ИНФЕРЕНС - не трогать

In [5]:
import cv2
import numpy as np
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo


def setup_model(model_weights_path, confidence_threshold=0.7):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"
    ))
    cfg.MODEL.WEIGHTS = model_weights_path
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = confidence_threshold
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    return DefaultPredictor(cfg)



def main(input_video: str, output_video: str, weights_path: str):
    # Инициализация модели и потоков
    predictor = setup_model(weights_path)
    cap = cv2.VideoCapture(input_video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Инференс
        outputs = predictor(frame)
        masks = outputs["instances"].pred_masks.cpu().numpy()
        print(masks)
        # Наложение масок на кадр
        for mask in masks:
            overlay = np.zeros_like(frame, dtype=np.uint8)
            overlay[mask] = (0, 255, 0)
            frame = cv2.addWeighted(frame, 1.0, overlay, 0.5, 0)
        # Запись кадра
        out.write(frame)

    cap.release()
    out.release()



In [ ]:
import cv2
import torch
import numpy as np
from typing import List, Tuple, Dict
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo


def setup_model(model_weights_path, confidence_threshold=0.7):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"
    ))
    cfg.MODEL.WEIGHTS = model_weights_path
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = confidence_threshold
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    return DefaultPredictor(cfg)
    
def get_homography(src_pts: np.ndarray, dst_pts: np.ndarray) -> np.ndarray:
    """Возвращает матрицу гомографии из src_pts в dst_pts"""
    H, _ = cv2.findHomography(src_pts, dst_pts)
    return H


def detect_crossing(
    bottom_m: Tuple[float, float],
    centroid_m: Tuple[float, float],
    history_centroids: List[Tuple[float, float]],
    last_sign: float,
    initial_recorded: bool,
    y_thresh: float = 0.5
) -> Tuple[bool, str]:
    """
    Проверяет, пересекло ли колесо границу:
    - initial_inside: если в первых кадрах уже внутри
    - cross_down: при переходе sign>0 -> sign<=0
    - y_jump: резкий перепад y > y_thresh
    Возвращает (crossing, reason).
    """
    xm_b, ym_b = bottom_m
    xm_c, ym_c = centroid_m
    dy = 4.5 - ym_b
    sign = np.sign(dy)
    crossing = False
    reason = None

    # initial inside
    if not initial_recorded and sign <= 0:
        crossing = True
        reason = "initial_inside"
    # cross down
    elif last_sign is not None and last_sign > 0 and sign <= 0:
        crossing = True
        reason = "cross_down"
    # jump in y
    elif last_sign is not None and abs(ym_b - (4.5 - last_sign * 0)) > y_thresh:
        crossing = True
        reason = "y_jump"

    # uniqueness by centroid
    if crossing:
        dists = [np.hypot(xm_c - cx, ym_c - cy) for cx, cy in history_centroids]
        if dists and min(dists) <= y_thresh:
            crossing = False  # слишком близко к уже зарегистрированному
    return crossing, reason


def draw_overlay(
    frame: np.ndarray,
    mask: np.ndarray,
    bottom_pixel: Tuple[int, int],
    centroid_pixel: Tuple[int, int],
    bottom_m: Tuple[float, float],
    reason: str = None
) -> np.ndarray:
    """Добавляет на кадр полупрозрачную маску, точки и метки события"""
    bx, by = bottom_pixel
    cx, cy = centroid_pixel
    xm_b, ym_b = bottom_m

    # mask overlay
    overlay = frame.copy()
    overlay[mask] = (0, 255, 0)
    frame = cv2.addWeighted(frame, 1.0, overlay, 0.4, 0)

    # bottom and centroid points
    cv2.circle(frame, (bx, by), 6, (0, 0, 255), -1)
    cv2.circle(frame, (int(cx), int(cy)), 6, (255, 0, 0), -1)

    # event label
    if reason:
        label = f"Wheel@{xm_b:.2f}m,{ym_b:.2f}m[{reason}]"
        cv2.putText(
            frame, label, (bx, by - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2
        )
    return frame


def main(
    input_video: str,
    output_video: str,
    weights_path: str,
    target_fps: float = None
):
    # 1. Инициализация
    predictor = setup_model(weights_path)
    cap = cv2.VideoCapture(input_video)
    fps_in = cap.get(cv2.CAP_PROP_FPS)
    w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 2. Настройка FPS и пропуска кадров
    if target_fps and 0 < target_fps < fps_in:
        skip = int(round(fps_in / target_fps))
        fps_out = target_fps
    else:
        skip = 1
        fps_out = fps_in
    writer = cv2.VideoWriter(
        output_video, cv2.VideoWriter_fourcc(*"XVID"), fps_out, (w, h)
    )

    # 3. Гомография
    src_pts = np.array([
        [1271.701, 246.366],
        [442.800,  366.435],
        [1435.023, 816.563],
        [1919.062, 468.274]
    ], dtype=float)
    dst_pts = np.array([
        [0.0, 0.0],
        [0.0, 4.5],
        [3.2, 4.5],
        [3.2, 0.0]
    ], dtype=float)
    H = get_homography(src_pts, dst_pts)
    contour = src_pts.astype(int).reshape(-1,1,2)

    # 4. Состояние
    events: List[Tuple[int,float,float,str]] = []
    history_centroids: List[Tuple[float,float]] = []
    last_sign = None
    initial_recorded = False

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1
        if frame_idx % skip != 0:
            continue

        # рисуем платформу
        cv2.polylines(frame, [contour], True, (255,0,0), 2)

        outputs = predictor(frame)
        masks = outputs["instances"].pred_masks.cpu().numpy()

        for m in masks:
            ys, xs = np.where(m)
            if xs.size == 0:
                continue
            # пиксельная геометрия
            idx_b = ys.argmax()
            bx, by = int(xs[idx_b]), int(ys[idx_b])
            cx, cy = float(xs.mean()), float(ys.mean())

            # метрическая геометрия
            xm_b, ym_b = cv2.perspectiveTransform(
                np.array([[[bx, by]]], float), H)[0,0]
            xm_c, ym_c = cv2.perspectiveTransform(
                np.array([[[cx, cy]]], float), H)[0,0]

            # проверка пересечения
            cross, reason = detect_crossing(
                (xm_b, ym_b), (xm_c, ym_c),
                history_centroids, last_sign, initial_recorded
            )
            if cross:
                events.append((frame_idx, xm_b, ym_b, reason))
                history_centroids.append((xm_c, ym_c))
                if reason == "initial_inside":
                    initial_recorded = True

            last_sign = np.sign(4.5 - ym_b)

            # отрисовка
            frame = draw_overlay(
                frame, m, (bx, by), (cx, cy), (xm_b, ym_b), reason
            )

        writer.write(frame)

    cap.release()
    writer.release()

    # 5. Вывод
    print("Detected wheel events:")
    for f, x, y, r in events:
        print(f"Frame {f}: bottom at x={x:.2f}m, y={y:.2f}m, reason={r}")
    print(f"Total wheels detected: {len(events)}")

In [39]:
import cv2
import numpy as np
from typing import List, Tuple, Dict
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from scipy.optimize import linear_sum_assignment

# Параметры трекинга
MAX_DIST = 5       # метров: макс. дистанция для связывания
MAX_MISSED = 100         # кадров: макс. пропусков
DIRECTION_PENALTY = 1.0  # штраф за изменение направления
KERNEL_SIZE = (3,3)    # морфология


def setup_model(weights_path: str, score_thresh: float = 0.5) -> DefaultPredictor:
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"
    ))
    cfg.MODEL.WEIGHTS = weights_path
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = score_thresh
    cfg.MODEL.DEVICE = "cuda"
    return DefaultPredictor(cfg)


def get_homography(src_pts: np.ndarray, dst_pts: np.ndarray) -> np.ndarray:
    H, _ = cv2.findHomography(src_pts, dst_pts)
    return H


def box_iou(boxA: Tuple[float,float,float,float], boxB: Tuple[float,float,float,float]) -> float:
    xA = max(boxA[0], boxB[0]); yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2]); yB = min(boxA[3], boxB[3])
    interW = max(0, xB - xA); interH = max(0, yB - yA)
    inter = interW * interH
    areaA = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    areaB = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])
    union = areaA + areaB - inter
    return inter / union if union > 0 else 0.0


def update_tracks(
    detections: List[Dict],
    tracks: List[Dict]
) -> Dict[int,int]:
    numT, numD = len(tracks), len(detections)
    # build cost matrix
    if numT > 0 and numD > 0:
        cost = np.zeros((numT, numD), dtype=float)
        for i, tr in enumerate(tracks):
            last_cx, last_cy = tr['history'][-1]
            dir_prev = tr.get('dir', None)
            for j, det in enumerate(detections):
                cx, cy = det['centroid']
                # distance
                dist = np.hypot(cx - last_cx, cy - last_cy)
                # IoU penalty
                iou = box_iou(det['bbox'], tr['bbox_last'])
                penalty = 1 - iou
                # direction penalty
                if dir_prev is not None and np.linalg.norm(dir_prev) > 0:
                    vec = np.array([cx - last_cx, cy - last_cy])
                    vn = vec / (np.linalg.norm(vec) + 1e-6)
                    dp = np.dot(dir_prev, vn)
                    if dp < 0:
                        penalty += DIRECTION_PENALTY
                cost[i, j] = dist + penalty
        row_idx, col_idx = linear_sum_assignment(cost)
    else:
        row_idx = np.array([], dtype=int)
        col_idx = np.array([], dtype=int)
    assignments = {}
    used_t, used_d = set(), set()
    # assign existing
    for i, j in zip(row_idx, col_idx):
        if cost[i, j] < MAX_DIST:
            tr = tracks[i]
            det = detections[j]
            tr['history'].append(det['centroid'])
            tr['bbox_last'] = det['bbox']
            # update direction
            dx, dy = np.subtract(tr['history'][-1], tr['history'][-2])
            tr['dir'] = (dx, dy) / (np.hypot(dx, dy) + 1e-6)
            tr['missed'] = 0
            assignments[j] = tr['id']
            used_t.add(i); used_d.add(j)
    # create new
    for j, det in enumerate(detections):
        if j not in used_d:
            new_id = len(tracks) + 1
            color = tuple(int(c) for c in np.random.randint(0,256,3))
            tracks.append({
                'id': new_id,
                'history': [det['centroid']],
                'bbox_last': det['bbox'],
                'dir': None,
                'missed': 0,
                'color': color
            })
            assignments[j] = new_id
    # handle missed
    for i, tr in enumerate(tracks):
        if i not in used_t:
            tr['missed'] += 1
            if tr['missed'] <= MAX_MISSED:
                tr['history'].append(tr['history'][-1])
    return assignments


def detect_crossing(
    bottom_m: Tuple[float,float],
    history_bottoms: List[float],
    initial_recorded: bool,
    y_thresh: float = 0.5
) -> Tuple[bool,str]:
    ym = bottom_m[1]
    dy = 4.5 - ym
    sign = np.sign(dy)
    crossing, reason = False, None
    if not initial_recorded and sign <= 0:
        crossing, reason = True, 'initial_inside'
    elif history_bottoms:
        last = history_bottoms[-1]
        sign_last = np.sign(4.5 - last)
        if sign_last > 0 and sign <= 0:
            crossing, reason = True, 'cross_down'
        elif len(history_bottoms) >= 5:
            avg_prev = np.mean(history_bottoms[-5:-1])
            if abs(ym - avg_prev) > y_thresh:
                crossing, reason = True, 'y_jump'
    return crossing, reason


def draw_overlay(
    frame: np.ndarray,
    mask: np.ndarray,
    bottom_px: Tuple[int,int],
    centroid_px: Tuple[float,float],
    bottom_m: Tuple[float,float],
    reason: str,
    track: Dict,
    H_inv: np.ndarray
) -> np.ndarray:
    # clean mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, KERNEL_SIZE)
    mask_clean = cv2.morphologyEx(mask.astype(np.uint8), cv2.MORPH_OPEN, kernel)
    bx, by = bottom_px
    cx, cy = map(int, centroid_px)
    xm, ym = bottom_m
    # overlay
    ov = frame.copy()
    ov[mask_clean.astype(bool)] = (0,255,0)
    frame = cv2.addWeighted(frame,1,ov,0.4,0)
    # points
    cv2.circle(frame, (bx,by), 5, (0,0,255), -1)
    cv2.circle(frame, (cx,cy), 5, (255,0,0), -1)
    # track line
    pts = []
    for wx, wy in track['history']:
        px = cv2.perspectiveTransform(np.array([[[wx, wy]]],float), H_inv)[0,0]
        pts.append((int(px[0]), int(px[1])))
    for i in range(1, len(pts)):
        cv2.line(frame, pts[i-1], pts[i], track['color'], 2)
    cv2.putText(frame, f"ID:{track['id']}", (bx+5, by-15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, track['color'], 2)
    if reason:
        cv2.putText(frame, f"{reason}@{xm:.2f},{ym:.2f}", (bx,by-25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 2)
    return frame


def main(
    input_video: str,
    output_video: str,
    weights_path: str,
    target_fps: float = None
):
    predictor = setup_model(weights_path)
    cap = cv2.VideoCapture(input_video)
    fps_in = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)); h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # setup FPS
    if target_fps and 0 < target_fps < fps_in:
        skip = int(round(fps_in / target_fps)); fps_out = target_fps
    else:
        skip, fps_out = 1, fps_in
    writer = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'XVID'), fps_out, (w,h))
    # homography and polygon
    src_pts = np.array([[1271.701,246.366], [442.8,366.435], [1435.023,816.563], [1919.062,468.274]], float)
    dst_pts = np.array([[0,0], [0,4.5], [3.2,4.5], [3.2,0]], float)
    H = get_homography(src_pts, dst_pts)
    H_inv, _ = cv2.findHomography(dst_pts, src_pts)
    contour = src_pts.astype(int).reshape(-1,1,2)
    # state
    tracks: List[Dict] = []
    events: List[Tuple[int,float,float,str,int]] = []
    history_bottoms: List[float] = []
    initial_recorded = False
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1
        if frame_idx % skip != 0:
            continue

        # draw platform
        cv2.polylines(frame, [contour], True, (255,0,0), 2)

        # inference
        outputs = predictor(frame)
        masks = outputs['instances'].pred_masks.cpu().numpy()
        boxes = outputs['instances'].pred_boxes.tensor.cpu().numpy()

        detections = []
        bottoms_px = []
        for m, b in zip(masks, boxes):
            # clean mask
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, KERNEL_SIZE)
            m_clean = cv2.morphologyEx(m.astype(np.uint8), cv2.MORPH_OPEN, kernel)
            ys, xs = np.where(m_clean)
            if xs.size == 0:
                continue
            idx = ys.argmax()
            bx, by = int(xs[idx]), int(ys[idx])
            cx, cy = float(xs.mean()), float(ys.mean())
            # world coords
            bm = cv2.perspectiveTransform(np.array([[[bx,by]]],float), H)[0,0]
            cm = cv2.perspectiveTransform(np.array([[[cx,cy]]],float), H)[0,0]
            detections.append({'centroid': (cm[0], cm[1]),
                               'bbox': tuple(b.tolist())})
            bottoms_px.append((bx, by, bm))

        # track update
        assign = update_tracks(detections, tracks)

        # process each detection
        for i, det in enumerate(detections):
            bx, by, bm = bottoms_px[i]
            tid = assign[i]
            track = next(t for t in tracks if t['id'] == tid)
            cross, reason = detect_crossing(bm, history_bottoms, initial_recorded)
            if cross:
                events.append((frame_idx, bm[0], bm[1], reason, tid))
                if reason == 'initial_inside':
                    initial_recorded = True
            history_bottoms.append(bm[1])
            frame = draw_overlay(frame, masks[i], (bx,by), det['centroid'], bm, reason, track, H_inv)

        writer.write(frame)

    cap.release()
    writer.release()

    # output log
    print("Detected events:")
    for f, x, y, reason, tid in events:
        print(f"Frame {f}: x={x:.2f}m, y={y:.2f}m, reason={reason}, ID={tid}")
    print(f"Total wheels: {len({e[4] for e in events})}")

In [42]:
main('/kaggle/input/end-videos/2test.avi','out.avi', '/kaggle/input/end/other/default/1/model_best.pth' )

Detected events:
Frame 569: x=2.60m, y=0.42m, reason=y_jump, ID=1
Frame 570: x=2.68m, y=0.76m, reason=y_jump, ID=1
Frame 571: x=2.71m, y=0.92m, reason=y_jump, ID=1
Frame 572: x=2.67m, y=0.84m, reason=y_jump, ID=1
Frame 573: x=2.70m, y=1.05m, reason=y_jump, ID=1
Frame 581: x=2.78m, y=1.75m, reason=y_jump, ID=1
Frame 582: x=2.79m, y=1.98m, reason=y_jump, ID=1
Frame 584: x=2.85m, y=2.20m, reason=y_jump, ID=1
Frame 589: x=2.37m, y=1.04m, reason=y_jump, ID=1
Frame 592: x=2.62m, y=2.51m, reason=y_jump, ID=1
Frame 593: x=2.67m, y=2.72m, reason=y_jump, ID=1
Frame 594: x=2.58m, y=2.52m, reason=y_jump, ID=1
Frame 595: x=2.68m, y=3.00m, reason=y_jump, ID=1
Frame 596: x=2.64m, y=3.14m, reason=y_jump, ID=1
Frame 597: x=2.64m, y=3.21m, reason=y_jump, ID=1
Frame 604: x=2.64m, y=3.86m, reason=y_jump, ID=1
Frame 604: x=2.74m, y=-1.67m, reason=y_jump, ID=2
Frame 605: x=2.75m, y=-1.42m, reason=y_jump, ID=2
Frame 606: x=2.64m, y=4.25m, reason=y_jump, ID=1
Frame 606: x=2.79m, y=-1.18m, reason=y_jump, ID=2


In [8]:
polygon = np.array([
        [464.691, 371.239],
        [1465.731, 795.823],
        [1919.063, 465.953],
        [1919.063, 181.809],
        [1879.392, 169.833]
    ], dtype=float)

main(
        video_path="/kaggle/input/videos/testup1.avi",
        weights_path="/kaggle/input/end/other/default/1/model_best.pth",
        polygon=polygon,
        px2m=args.scale,
        target_fps=args.fps
    )

NameError: name 'analyze_video' is not defined

In [14]:
import cv2
import numpy as np
from typing import List, Tuple, Dict
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo
from scipy.optimize import linear_sum_assignment

# Параметры
MAX_DIST = 10       # макс. дистанция для связывания
MAX_MISSED = 180         # кадров макс. пропусков
DIRECTION_PENALTY = 0.9  # штраф за изменение направления
KERNEL_SIZE = (3,3)    # морфология

MODE_CONFIG = {
    'start': {
        'weights': '/kaggle/input/start/other/default/1/model_best.pth',
        'score_thresh': 0.7
    },
    'end': {
        'weights': '/kaggle/input/end/other/default/1/model_best.pth',
        'score_thresh': 0.7
    }
}

# Параметры зон по режимам
ZONE_PARAMS = {
    'end': {
        'src_pts': np.array([[1271.701, 246.366], [442.8, 366.435], [1435.023, 816.563], [1919.062, 468.274]], float),
        'dst_pts': np.array([[0, 0], [0, 4.5], [3.2, 4.5], [3.2, 0]], float),
        'length': 4.5,
        'width': 3.2
    },
    'start': {
        'src_pts': np.array([[852.967, 698.86], [69.749, 796.118], [316.449, 1079.058], [1434.073, 890.692]], float),
        'dst_pts': np.array([[0, 0], [0, 3.75], [3.2, 3.75], [3.2, 0]], float),
        'length': 3.75,
        'width': 3.2
    }
}


def setup_model(weights: str, score_thresh: float) -> DefaultPredictor:
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"
    ))
    cfg.MODEL.WEIGHTS = weights
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = score_thresh
    cfg.MODEL.DEVICE = 'cuda'
    return DefaultPredictor(cfg)


def get_homography(src_pts: np.ndarray, dst_pts: np.ndarray) -> np.ndarray:
    H, _ = cv2.findHomography(src_pts, dst_pts)
    return H


def box_iou(boxA: Tuple[float, float, float, float], boxB: Tuple[float, float, float, float]) -> float:
    xA, yA = max(boxA[0], boxB[0]), max(boxA[1], boxB[1])
    xB, yB = min(boxA[2], boxB[2]), min(boxA[3], boxB[3])
    interW, interH = max(0, xB - xA), max(0, yB - yA)
    inter = interW * interH
    areaA = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    areaB = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])
    union = areaA + areaB - inter
    return inter / union if union > 0 else 0.0


def update_tracks(
    detections: List[Dict],
    tracks: List[Dict]
) -> Dict[int, int]:
    numT, numD = len(tracks), len(detections)
    if numT > 0 and numD > 0:
        cost = np.zeros((numT, numD), dtype=float)
        for i, tr in enumerate(tracks):
            lx, ly = tr['history'][-1]
            dir_prev = tr.get('dir', None)
            for j, det in enumerate(detections):
                cx, cy = det['centroid']
                dist = np.hypot(cx - lx, cy - ly)
                iou = box_iou(det['bbox'], tr['bbox_last'])
                penalty = 1 - iou
                if dir_prev is not None:
                    vec = np.array([cx - lx, cy - ly])
                    vnorm = vec / (np.linalg.norm(vec) + 1e-6)
                    if np.dot(dir_prev, vnorm) < 0:
                        penalty += DIRECTION_PENALTY
                cost[i, j] = dist + penalty
        row_idx, col_idx = linear_sum_assignment(cost)
    else:
        row_idx = np.array([], dtype=int)
        col_idx = np.array([], dtype=int)
    assignments, used_t, used_d = {}, set(), set()
    # assign matched
    for i, j in zip(row_idx, col_idx):
        if cost[i, j] < MAX_DIST:
            tr = tracks[i]; det = detections[j]
            tr['history'].append(det['centroid'])
            tr['bbox_last'] = det['bbox']
            dx, dy = np.subtract(tr['history'][-1], tr['history'][-2])
            tr['dir'] = (dx, dy) / (np.hypot(dx, dy) + 1e-6)
            tr['missed'] = 0
            assignments[j] = tr['id']; used_t.add(i); used_d.add(j)
    # create new tracks
    for j, det in enumerate(detections):
        if j not in used_d:
            new_id = len(tracks) + 1
            tracks.append({
                'id': new_id,
                'history': [det['centroid']],
                'bbox_last': det['bbox'],
                'dir': None,
                'missed': 0,
                'color': tuple(int(c) for c in np.random.randint(0, 256, 3))
            })
            assignments[j] = new_id
    # handle missed
    for i, tr in enumerate(tracks):
        if i not in used_t:
            tr['missed'] += 1
            if tr['missed'] <= MAX_MISSED:
                tr['history'].append(tr['history'][-1])
    return assignments


def detect_crossing(
    bottom_m: Tuple[float, float],
    history_bottoms: List[float],
    initial_recorded: bool,
    length: float,
    y_thresh: float = 0.5
) -> Tuple[bool, str]:
    ym = bottom_m[1]
    sign = np.sign(length - ym)
    crossing, reason = False, None
    if not initial_recorded and sign <= 0:
        crossing, reason = True, 'initial_inside'
    elif history_bottoms:
        last = history_bottoms[-1]
        sign_last = np.sign(length - last)
        if sign_last > 0 and sign <= 0:
            crossing, reason = True, 'cross_down'
        elif len(history_bottoms) >= 5:
            avg_prev = np.mean(history_bottoms[-5:-1])
            if abs(ym - avg_prev) > y_thresh:
                crossing, reason = True, 'y_jump'
    return crossing, reason


def draw_overlay(
    frame, mask, bottom_px, centroid_px, bottom_m, reason, track, H_inv
) -> np.ndarray:
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, KERNEL_SIZE)
    m_clean = cv2.morphologyEx(mask.astype(np.uint8), cv2.MORPH_OPEN, kernel)
    bx, by = bottom_px; cx, cy = map(int, centroid_px)
    xm, ym = bottom_m
    ov = frame.copy(); ov[m_clean.astype(bool)] = (0,255,0)
    frame = cv2.addWeighted(frame,1,ov,0.4,0)
    cv2.circle(frame,(bx,by),5,(0,0,255),-1)
    cv2.circle(frame,(cx,cy),5,(255,0,0),-1)
    pts = []
    for frm, wx, wy in track['world_pixel_history']:
        pts.append((wx, wy))
    for i in range(1,len(pts)):
        cv2.line(frame,pts[i-1],pts[i],track['color'],2)
    cv2.putText(frame,f"ID:{track['id']}",(bx+5,by-15),cv2.FONT_HERSHEY_SIMPLEX,0.5,track['color'],2)
    if reason:
        cv2.putText(frame,f"{reason}@{xm:.2f},{ym:.2f}",(bx,by-25),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,255),2)

    bx, by = bottom_px
    xm, ym = bottom_m

    # 1) Рисуем нижнюю точку
    cv2.circle(frame, (bx,by), 5, (0,0,255), -1)

    # 2) Выбираем цвет текста по фазе движения
    if reason == 'initial_inside':
        text_color = (0,255,0)     # до остановки – зелёный
    elif reason in ('cross_down','y_jump'):
        text_color = (0,165,255)   # во время входа/пересечения – оранжевый
    else:
        text_color = (255,0,0)     # после – синий

    # 3) Подписываем пиксельные и мировые координаты
    label = f"px:({bx},{by})  m:({xm:.2f},{ym:.2f})"
    cv2.putText(
        frame, label, (bx+10, by-10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 1
    )

    return frame
    return frame


def main(
    input_video: str,
    output_video: str,
    mode: str,
    target_fps: float = None
):
    # конфиг в зависимости от режима
    mcfg = MODE_CONFIG[mode]
    zcfg = ZONE_PARAMS[mode]
    predictor = setup_model(
        mcfg['weights'], mcfg['score_thresh']
    )
    src_pts, dst_pts = zcfg['src_pts'], zcfg['dst_pts']
    length, width = zcfg['length'], zcfg['width']

    cap = cv2.VideoCapture(input_video)
    fps_in = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)); h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    if target_fps and 0 < target_fps < fps_in:
        skip = int(round(fps_in / target_fps)); fps_out = target_fps
    else:
        skip, fps_out = 1, fps_in
    writer = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'XVID'), fps_out, (w, h))

    # гомография полигона детекции
    H = get_homography(src_pts, dst_pts)
    H_inv, _ = cv2.findHomography(dst_pts, src_pts)
    contour = src_pts.astype(int).reshape(-1, 1, 2)

    tracks, events, history_bottoms = [], [], []
    initial_recorded = False
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret: break
        frame_idx += 1
        if frame_idx % skip != 0: continue

        # зона детекции
        cv2.polylines(frame, [contour], True, (255, 0, 0), 2)

        # гет предикт
        outputs = predictor(frame)
        masks = outputs['instances'].pred_masks.cpu().numpy()
        boxes = outputs['instances'].pred_boxes.tensor.cpu().numpy()

        detections, bottoms_px = [], []
        for m, b in zip(masks, boxes):
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, KERNEL_SIZE)
            m_clean = cv2.morphologyEx(m.astype(np.uint8), cv2.MORPH_OPEN, kernel)
            ys, xs = np.where(m_clean)
            if xs.size == 0: continue
            idx = ys.argmax(); bx, by = int(xs[idx]), int(ys[idx])
            cx, cy = float(xs.mean()), float(ys.mean())
            bm = cv2.perspectiveTransform(np.array([[[bx, by]]], float), H)[0, 0]
            cm = cv2.perspectiveTransform(np.array([[[cx, cy]]], float), H)[0, 0]
            detections.append({'centroid': (cm[0], cm[1]), 'bbox': tuple(b.tolist())})
            bottoms_px.append((bx, by, bm))

        assign = update_tracks(detections, tracks)

        for i, det in enumerate(detections):
            bx, by, bm = bottoms_px[i]
            tid = assign[i]
            tr = next(t for t in tracks if t['id'] == tid)
            tr.setdefault('pixel_history', []).append((frame_idx, bx, by))
            tr.setdefault('world_pixel_history', []).append((frame_idx, bx, by))
            tr.setdefault('world_bottom_history', []).append((frame_idx, bm[0], bm[1]))

            cross, reason = detect_crossing(bm, history_bottoms, initial_recorded, length)
            if cross:
                events.append((frame_idx, bm[0], bm[1], reason, tid))
                if reason == 'initial_inside': initial_recorded = True
            history_bottoms.append(bm[1])

            frame = draw_overlay(frame, masks[i], (bx, by), det['centroid'], bm, reason, tr, H_inv)

        writer.write(frame)

    cap.release(); writer.release()

    # главная дорожка трекера
    first_pos = {t['id']: t['pixel_history'][0][1] for t in tracks}
    ref_id = max(first_pos, key=first_pos.get) if mode == 'start' else min(first_pos, key=first_pos.get)
    ref_track = next(t for t in tracks if t['id'] == ref_id)

    # триггер стопа
    stop_thr = int(2 * fps_out)
    stationary = 0; stop_frame = ref_track['pixel_history'][-1][0]
    prev_x = None
    for frm, px, _ in ref_track['pixel_history']:
        if prev_x is not None and abs(px - prev_x) <= 10:
            stationary += 1
            if stationary >= stop_thr:
                stop_frame = frm; break
        else:
            stationary = 0
        prev_x = px

    # пройденный путь в м до остановки
    y_start = ref_track['world_bottom_history'][0][2]
    y_stop = next(y for f, x, y in ref_track['world_bottom_history'] if f == stop_frame)
    traveled = abs(y_stop - y_start)

    # точка остановки
    x_stop = next(x for f, x, y in ref_track['world_bottom_history'] if f == stop_frame)

    # Расчет фронтального расстояния до переднего края зоны
    if mode == 'end':
        front_dist = length - y_stop  # расстояние от переднего края (y=0)
        side_dist = width - x_stop   # расстояние до левого борта
    else:
        front_dist = (width - y_stop)  # до заднего края
        side_dist = (length - x_stop)    # до правого борта

    print("События:")
    for f, x, y, reason, tid in events:
        print(f"Frame {f}: x={x:.2f}m, y={y:.2f}m, reason={reason}, ID={tid}")
    print(f"Total wheels: {len({e[4] for e in events})}")
    print(f"Reference track ID: {ref_id}")
    print(f"Traveled distance: {traveled:.2f} m")
    print(f"Front distance to zone edge: {front_dist:.2f} m")
    print(f"Side distance to zone border: {side_dist:.2f} m")
    return events

In [7]:
events_start = main('/kaggle/input/start_videos/other/default/1/1test.avi','out.avi', 'start' )

/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


События:
Frame 634: x=1.46m, y=2.35m, reason=y_jump, ID=1
Frame 637: x=1.72m, y=2.34m, reason=y_jump, ID=1
Frame 643: x=2.38m, y=2.31m, reason=y_jump, ID=1
Frame 655: x=0.97m, y=3.05m, reason=y_jump, ID=2
Frame 675: x=3.88m, y=2.16m, reason=y_jump, ID=1
Frame 678: x=2.34m, y=3.10m, reason=y_jump, ID=2
Frame 683: x=4.12m, y=1.48m, reason=y_jump, ID=1
Frame 696: x=3.35m, y=2.97m, reason=y_jump, ID=2
Frame 697: x=3.32m, y=2.97m, reason=y_jump, ID=2
Total wheels: 2
Reference track ID: 2
Traveled distance: 0.00 m
Front distance to zone edge: 0.18 m
Side distance to zone border: 0.35 m


In [15]:
events_end = main('/kaggle/input/end-videos/5test.avi','out1.avi', 'end' )

События:
Frame 295: x=2.78m, y=0.67m, reason=y_jump, ID=1
Frame 296: x=2.78m, y=0.78m, reason=y_jump, ID=1
Frame 297: x=2.80m, y=0.81m, reason=y_jump, ID=1
Frame 725: x=2.28m, y=-0.49m, reason=y_jump, ID=1
Frame 727: x=2.73m, y=1.92m, reason=y_jump, ID=1
Frame 728: x=2.78m, y=2.07m, reason=y_jump, ID=1
Frame 729: x=2.78m, y=2.21m, reason=y_jump, ID=1
Frame 730: x=2.79m, y=2.14m, reason=y_jump, ID=1
Frame 735: x=2.57m, y=1.21m, reason=y_jump, ID=1
Frame 736: x=2.58m, y=1.26m, reason=y_jump, ID=1
Frame 737: x=2.54m, y=1.27m, reason=y_jump, ID=1
Frame 737: x=2.17m, y=0.18m, reason=y_jump, ID=2
Frame 738: x=2.13m, y=-0.09m, reason=y_jump, ID=2
Frame 739: x=2.76m, y=2.17m, reason=y_jump, ID=1
Frame 740: x=2.74m, y=2.16m, reason=y_jump, ID=1
Frame 741: x=2.70m, y=1.98m, reason=y_jump, ID=1
Frame 742: x=2.68m, y=2.63m, reason=y_jump, ID=1
Frame 743: x=2.75m, y=2.91m, reason=y_jump, ID=1
Frame 743: x=2.09m, y=0.30m, reason=y_jump, ID=2
Frame 744: x=2.77m, y=3.01m, reason=y_jump, ID=1
Frame 744